# Julia Basics

Pablo Winant / QuantEcon

### What is Julia

-   developped at MIT on top of **opensource** technologies
    -   linux / git / llvm
-   syntax inspired by Matlab but:
    -   more consistent
    -   lots of features from high level languages
-   everything is JIT-compiled
    -   interpreted vs compiled treadeoff
    -   -\> very fast
    -   most of the base library is written in Julia
-   opensource/free + vibrant community

Some useful links from QuantEcon:

-   [Julia
    cheatsheet](https://cheatsheets.quantecon.org/julia-cheatsheet.html)
-   [Julia-Matlab
    comparison](https://cheatsheets.quantecon.org/index.html)
-   [Vectors, arrays and
    matrices](https://lectures.quantecon.org/jl/julia_arrays.html)

Excellent resources at: [julialang](https://julialang.org/learning/)

-   checkout JuliaAcademy, it’s free
-   ongoing [MOOC](https://computationalthinking.mit.edu/Spring21/) at
    MIT

### an example of what you shouldn’t do in Matlab

How I learnt: interpreted code is slow, so vectorize your code.

In [1]:
function stupid_loop(I,J,K)
    t = 0.0
    for i=1:I
        for j=1:J
            for k = 1:K
                t += 1.0
            end        
        end
    end
    return t
end
@time [ stupid_loop(1000,1000,i) for i =1:10]

  0.118460 seconds (46.57 k allocations: 2.332 MiB, 37.47% compilation time)


10-element Vector{Float64}:
 1.0e6
 2.0e6
 3.0e6
 4.0e6
 5.0e6
 6.0e6
 7.0e6
 8.0e6
 9.0e6
 1.0e7

In [2]:
stupid_loop(1000,1000,1000)

1.0e9

Code is translated to LLVM code then to instructions for the processor.
Note that processor instructions are shorter than LLVM code.

In [3]:
@code_llvm stupid_loop(10,10,10)

; Function Signature: stupid_loop(Int64, Int64, Int64)
;  @ In[1]:1 within `stupid_loop`
; Function Attrs: uwtable
define double @julia_stupid_loop_3341(i64 signext %"I::Int64", i64 signext %"J::Int64", i64 signext %"K::Int64") #0 {
top:
;  @ In[1] within `stupid_loop`
  %".I::Int64" = call i64 @llvm.smax.i64(i64 %"I::Int64", i64 0)
;  @ In[1]:3 within `stupid_loop`
; ┌ @ range.jl:917 within `iterate`
; │┌ @ range.jl:688 within `isempty`
; ││┌ @ operators.jl:425 within `>`
; │││┌ @ int.jl:83 within `<`
      %0 = icmp slt i64 %"I::Int64", 1
; └└└└
  br i1 %0, label %L85, label %L16.preheader

L16.preheader:                                    ; preds = %top
  %".J::Int64" = call i64 @llvm.smax.i64(i64 %"J::Int64", i64 0)
  %1 = icmp slt i64 %"J::Int64", 1
  %".K::Int64" = call i64 @llvm.smax.i64(i64 %"K::Int64", i64 0)
  %2 = icmp slt i64 %"K::Int64", 1
;  @ In[1]:4 within `stupid_loop`
  %or.cond = select i1 %1, i1 true, i1 %2
  br i1 %or.cond, label %L85, label %L16.preheader36

L16.p

In [4]:
@code_native stupid_loop(10,10,10)

	.text
	.file	"stupid_loop"
	.section	.rodata.cst8,"aM",@progbits,8
	.p2align	3, 0x0                          # -- Begin function julia_stupid_loop_3454
.LCPI0_0:
	.quad	0x3ff0000000000000              # double 1
	.section	.ltext,"axl",@progbits
	.globl	julia_stupid_loop_3454
	.p2align	4, 0x90
	.type	julia_stupid_loop_3454,@function
julia_stupid_loop_3454:                 # @julia_stupid_loop_3454
; Function Signature: stupid_loop(Int64, Int64, Int64)
; ┌ @ In[1]:1 within `stupid_loop`
	.cfi_startproc
# %bb.0:                                # %top
	#DEBUG_VALUE: stupid_loop:I <- $rcx
	#DEBUG_VALUE: stupid_loop:J <- $rdx
	#DEBUG_VALUE: stupid_loop:K <- $r8
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	rsi
	push	rdi
	.cfi_offset rdi, -32
	.cfi_offset rsi, -24
	xor	eax, eax
	vxorpd	xmm0, xmm0, xmm0
; │ @ In[1] within `stupid_loop`
	test	rcx, rcx
	cmovg	rax, rcx
; │ @ In[1]:3 within `stupid_loop`
	jle	.LBB0_13
# %bb.1:               

### Syntax Review

#### Variable assignment

Assignement operator is = (equality is ==, identity is ===)

In [5]:
# Assign the value 10 to the variable x
x = 10

10

In [6]:
x

10

In [7]:
2 == 3

false

In [8]:
# Variable names can have Unicode characters
# To get ϵ in the REPL, type \epsilon<TAB>
a = 20
α = 10
🐳 = 0.1
🦈 = 0.1 * 🐳
σ = 34
ϵ = 1e-4

0.0001

Default semantic is pass-by-reference:

In [9]:
a = [1,2,3,4]
b = a
a[1] = 10
b

4-element Vector{Int64}:
 10
  2
  3
  4

To work on a copy: `copy` or `deepcopy`

In [10]:
a = [1,2,3,4]
b = copy(a)
a[1]=10
b

4-element Vector{Int64}:
 1
 2
 3
 4

In [11]:
a .== b

4-element BitVector:
 0
 1
 1
 1

In [12]:
c = b

4-element Vector{Int64}:
 1
 2
 3
 4

In [13]:
b = [1,2,3,4]

4-element Vector{Int64}:
 1
 2
 3
 4

In [14]:
a .== b

4-element BitVector:
 0
 1
 1
 1

In [15]:
c === b

false

#### Basic types

In [16]:
# for any object `typeof` returns the type

typeof(a)

Vector{Int64} (alias for Array{Int64, 1})

In [17]:
[1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [18]:
typeof(randn(3,3))  == Array{Float64, 2}

true

#### Numbers

In [19]:
y = 2 + 2

4

In [20]:
-y

-4

In [21]:
0.34*23

7.82

In [22]:
3/4

0.75

In [23]:
3//4

3//4

In [24]:
3//4 + 2//3

17//12

In [25]:
typeof(3//4 + 2//3)

Rational{Int64}

In [26]:
# Scalar multiplication doesn't require *
3(4 - 2) 

6

In [27]:
x = 4
2*x + 2x^2

40

In [28]:
typeof(x)

Int64

In [29]:
sizeof(x)

8

In [30]:
typeof(10)

Int64

In [31]:
(big(100))//big(1000)

1//10

In [32]:
bitstring(10)

"0000000000000000000000000000000000000000000000000000000000001010"

#### Booleans

Equality

In [33]:
0 == 1

false

In [34]:
2 != 3

true

In [35]:
3 < 4

true

In [36]:
true == false

false

Identity

In [37]:
a = [34, 35]
b = [34, 35]
c = a

2-element Vector{Int64}:
 34
 35

In [38]:
c === a

true

In [39]:
b === a

false

Boolean operator

In [40]:
true && false

false

In [41]:
true || false

true

In [42]:
!true

false

In [43]:
a  = 2
b = 3

(a > b) && (factorial(100) > 10)

false

#### Strings

In [44]:
# Strings are written using double quotes
str = "This is a string"

"This is a string"

In [45]:
ch = '🦆' # this is a character

'🦆': Unicode U+1F986 (category So: Symbol, other)

In [46]:
# Strings can also contain Unicode characters
fancy_str = "α is a string"

"α is a string"

In [47]:
n = 10
println("Iteration : ", n)

Iteration : 10


In [48]:
# String interpolation using $
# The expression in parentheses is evaluated and the result is 
# inserted into the string
a = 2+2
"2 + 2 = $(a+1)"

"2 + 2 = 5"

In [49]:
println("It took me $(a) iterations")

It took me 4 iterations


In [50]:
# String concatenation using *
"hello" * "world"

"helloworld"

In [51]:
print("1")
print("2")
print("3")

123

In [52]:
println("1")
println("2")
println("3")

1
2
3


In [53]:
println("hello ", "world")

hello world


#### Arrays

Julia has one-dimensional arrays. They are also called Vector.

In [54]:
A = [1, 2]

2-element Vector{Int64}:
 1
 2

All elements have the type:

In [55]:
A = [1, 1.4]

2-element Vector{Float64}:
 1.0
 1.4

In [56]:
typeof(A) == Vector{Int64}

false

In [57]:
A''

2-element Vector{Float64}:
 1.0
 1.4

To get the size of an array:

In [58]:
length(A)

2

In [59]:
size(A)

(2,)

Arrays are *mutable*

In [60]:
A[1] = 10

10

In [61]:
A

2-element Vector{Float64}:
 10.0
  1.4

Julia has one-based indexing: you refer to the first element as 1
($\neq$ zero-based indexing in C or Python)

In [62]:
A[2]

1.4

Arrays are mutable and their size can be changed too:

In [63]:
push!(A, 29)
A

3-element Vector{Float64}:
 10.0
  1.4
 29.0

In [64]:
A

3-element Vector{Float64}:
 10.0
  1.4
 29.0

In [65]:
prepend!(A, 28)

4-element Vector{Float64}:
 28.0
 10.0
  1.4
 29.0

Two comments: - the `push!` operation is *fast* - `!` is a julia
convention to express the fact that `push!` mutates its first argument

In [66]:
["a", "b"]

2-element Vector{String}:
 "a"
 "b"

In [67]:
["a", 1]

2-element Vector{Any}:
  "a"
 1

#### tuples

In [68]:
size(A)  # is a tuple

(4,)

In [69]:
(5,)

(5,)

In [70]:
# you can create tuples with (,,,)
t = (1,2,3,4)

(1, 2, 3, 4)

In [71]:
t

(1, 2, 3, 4)

tuples differ from arrays in two ways: - they are immutable - they can
contain non-homogenous objects

In [72]:
t[1]

1

In [73]:
t[1] = 2

LoadError: MethodError: no method matching setindex!(::NTuple{4, Int64}, ::Int64, ::Int64)
The function `setindex!` exists, but no method is defined for this combination of argument types.

In [74]:
typeof((1, "1", [1]))

Tuple{Int64, String, Vector{Int64}}

2d arrays are also called matrices… and can be used for matrix
multiplications.

In [75]:
[3 4; 5 6]

2×2 Matrix{Int64}:
 3  4
 5  6

In [76]:
[ [3, 4];; [5, 6]] # concatenate along second dimension

2×2 Matrix{Int64}:
 3  5
 4  6

In [77]:
a1 = [1,2,3,4]
a2 = [1,2,3,4]  .+ 4
[a1 ;; a2]
cat(a1, a2; dims=2)

4×2 Matrix{Int64}:
 1  5
 2  6
 3  7
 4  8

In [78]:
b = [1 0.6 0]

1×3 Matrix{Float64}:
 1.0  0.6  0.0

In [79]:
B = [0.1 0.2 0.3; 4 5 6]

2×3 Matrix{Float64}:
 0.1  0.2  0.3
 4.0  5.0  6.0

Other ways to construct arrays:

In [80]:
# zero array
t = zeros(2,3)
t[1,2] = 23.2
t

2×3 Matrix{Float64}:
 0.0  23.2  0.0
 0.0   0.0  0.0

In [81]:
# zero array
t = zeros(Int64,2,3)
# t[1,2] = 23.2
t

2×3 Matrix{Int64}:
 0  0  0
 0  0  0

In [82]:
# random array (uniform distribution)
t= rand(3,3)
t

3×3 Matrix{Float64}:
 0.880679   0.556687  0.708702
 0.0630292  0.939843  0.897365
 0.483668   0.164147  0.566517

In [83]:
# random array (normal distribution)
t= randn(3,3)
t

3×3 Matrix{Float64}:
 -0.324902  -0.94895    0.859416
 -1.30637   -0.0500134  0.347269
  1.20322   -0.0124077  1.30564

Vectorized operations take a ., even comparisons (pointwise operations)

In [84]:
B = [1 2;3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [85]:
B*B

2×2 Matrix{Int64}:
  7  10
 15  22

In [86]:
B .* B

2×2 Matrix{Int64}:
 1   4
 9  16

In [87]:
f(x) = x^2+1

f (generic function with 1 method)

In [88]:
f(43)

1850

In [89]:
# [ f(e) for e in [1,2,3,4,5] ]
f.([1,2,3,4,5])

5-element Vector{Int64}:
  2
  5
 10
 17
 26

Elements are always accessed with square brackets:

In [90]:
B = [1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [91]:
You get element $B_{ij}$ with `B[i,j]`

LoadError: UndefVarError: `You` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [92]:
B[1,2]

2

You select a whole row/column with `:`

In [93]:
B[1,:]

3-element Vector{Int64}:
 1
 2
 3

In [94]:
B[:,1]

2-element Vector{Int64}:
 1
 4

In [95]:
B[:,1:2]

2×2 Matrix{Int64}:
 1  2
 4  5

In [96]:
B[:,1:end-1]

2×2 Matrix{Int64}:
 1  2
 4  5

#### Control flow

Conditions

In [97]:
x = 0
if x<0
    # block
    println("x is negative")
elseif (x > 0) # optional and unlimited
    println("x is positive")
else         # optional
    println("x is zero")
end

x is zero


While

In [98]:
i = 3
while i > 0
    println(i)
    i -= 1 # decrement
end

3
2
1


For loops: your iterate over any iterable object: - range `i1:i2` -
vector - tuple

In [99]:
# Iterate through ranges of numbers
for i ∈ (1:3)
    println(i)
end

1
2
3


In [100]:
# Iterate through arrays
cities = ["Boston", "New York", "Philadelphia"]
for city ∈ cities
    println(city)
end

Boston
New York
Philadelphia


In [101]:
cities

3-element Vector{String}:
 "Boston"
 "New York"
 "Philadelphia"

In [102]:
states = ["Massachussets", "New York", "Pennsylvania"]

3-element Vector{String}:
 "Massachussets"
 "New York"
 "Pennsylvania"

In [103]:
two_by_two_iterable = zip(cities, states)

zip(["Boston", "New York", "Philadelphia"], ["Massachussets", "New York", "Pennsylvania"])

In [104]:
typeof(two_by_two_iterable)

Base.Iterators.Zip{Tuple{Vector{String}, Vector{String}}}

In [105]:
collect(two_by_two_iterable)

3-element Vector{Tuple{String, String}}:
 ("Boston", "Massachussets")
 ("New York", "New York")
 ("Philadelphia", "Pennsylvania")

In [106]:
[two_by_two_iterable...]

3-element Vector{Tuple{String, String}}:
 ("Boston", "Massachussets")
 ("New York", "New York")
 ("Philadelphia", "Pennsylvania")

In [107]:
# Iterate through arrays of tuples using zip
for kw in zip(cities, states)
    println(kw)
end

("Boston", "Massachussets")
("New York", "New York")
("Philadelphia", "Pennsylvania")


In [108]:
# Iterate through arrays of tuples using zip
for (city, state) in zip(cities, states)
    println("City: $city | State: $state")
end

City: Boston | State: Massachussets
City: New York | State: New York
City: Philadelphia | State: Pennsylvania


In [109]:
# Iterate through arrays and their indices using enumerate
for (i, city) in enumerate(cities)
    println("City $i is $city")
end

City 1 is Boston
City 2 is New York
City 3 is Philadelphia


#### List comprehensions

In [110]:
[1:10 ...] # unpack operator

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [111]:
[i^2 for i  in 1:10] # collect with comprehension syntax

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [112]:
[i^2 for i=1:10000000 if mod(i,2)==0] ;

In [113]:
@time sum( [i^2 for i=1:10000000000 if mod(i,2)==0] )

LoadError: OutOfMemoryError()

In [ ]:
function fun()
    t = 0
    for  i=1:10000000000
        if mod(i,2)==0
            t += i^2
        end
    end
    return t
end

In [ ]:
@time fun()

In [ ]:
gen = (i^2 for i=1:10000000000 if mod(i,2)==0)

In [ ]:
@time sum(gen)

In [ ]:
## Named Tuples

In [ ]:
t = (;a=1,b=2,c=3)

In [ ]:
t[1] # indexed like tuple
# t[1] = 2 # immutable
t.a # access fields using names

In [ ]:
model = (;
    α = 0.3,
    β = 0.96
)

In [ ]:
merge(model, (;β=0.9, γ=0.2))

In [ ]:
# unpack values from a tuple

α = model[1]
β = model[2]

In [ ]:
# unpack values from a namedtuple

α = model.α
β = model.β

In [ ]:
# namedtuple unpacking

(;α, β) = model
α

### Data Types and multiple dispatch

#### Composite types

A **composite type** is a collection of named fields that can be treated
as a single value. They bear a passing resemblance to MATLAB structs.

All fields must be declared ahead of time. The double colon, `::`,
constrains a field to contain values of a certain type. This is optional
for any field.

In [ ]:
# Type definition with 4 fields
struct ParameterFree
    value  
    transformation  
    tex_label
    description 
end

In [ ]:
pf = ParameterFree("1", x->x^2, "\\sqrt{1+x^2}", ("a",1))

In [ ]:
pf.value

Two reasons to create structures: - syntactic shortcut (you access the
fields with .) - specify the types of the fields

In [ ]:
# Type definition
struct Parameter
    value ::Float64
    transformation ::Function # Function is a type!
    tex_label::String
    description::String
end

In [ ]:
p = Parameter("1", x->x^2, "\\sqrt{1+x^2}", ("a",1))

In [ ]:
p = Parameter(0.43, x->x^2, "\\sqrt{1+x^2}", "This is a description")

In [ ]:
p.value

When a type with $n$ fields is defined, a constructor (function that
creates an instance of that type) that takes $n$ ordered arguments is
automatically created. Additional constructors can be defined for
convenience.

In [ ]:
# Creating an instance of the Parameter type using the default
# constructor
β = Parameter(0.9, identity, "\\beta", "Discount rate")

In [ ]:
function Parameter(value)
    return Parameter(value, x->x, "x", "Anonymous")
end

In [ ]:
Parameter(0.4)

In [ ]:
Parameter(value, transformation, tex) = Parameter(value, transformation, tex, "no description")

In [ ]:
methods( Parameter )

In [ ]:
# Alternative constructors end with an appeal to the default
# constructor
function Parameter(value::Float64, tex_label::String)
    transformation = identity
    description = "No description available"
    return Parameter(value, transformation, tex_label, description)
end

α = Parameter(0.5, "\alpha")

Now the function `Parameter` has two different `methods` with different
signatures:

In [ ]:
methods(Parameter)

We have seen that a function can have several implementations, called
methods, for different number of arguments, or for different types of
arguments.

In [ ]:
fun(x::Int64, y::Int64) = x^3 + y

In [ ]:
fun(x::Float64, y::Int64) = x/2 + y

In [ ]:
fun(2, 2)

In [ ]:
fun(2.0, 2)

In [ ]:
α.tex_label

In [ ]:
# Access a particular field using .
α.value

In [ ]:
# Fields are modifiable and can be assigned to, like 
# ordinary variables
α.value = 0.75

### Mutable vs non mutable types

by default structures in Julia are non-mutable

In [ ]:
p.value = 3.0

In [ ]:
mutable struct Params
    x:: Float64
    y:: Float64
end

In [ ]:
pos = Params(0.4, 0.2)

In [ ]:
pos.x = 0.5

### Parameterized Types

**Parameterized types** are data types that are defined to handle values
identically regardless of the type of those values.

Arrays are a familiar example. An `Array{T,1}` is a one-dimensional
array filled with objects of any type `T` (e.g. `Float64`, `String`).

In [ ]:
# Defining a parametric point
struct Duple{T} # T is a parameter to the type Duple
    x::T
    y::T
end

In [ ]:
Duple(3, 3)

In [ ]:
Duple(3, -1.0)

In [ ]:
struct Truple{T}
    x::Duple{T}
    z::T
end

This single declaration defines an unlimited number of new types:
`Duple{String}`, `Duple{Float64}`, etc. are all immediately usable.

In [ ]:
sizeof(3.0)

In [ ]:
sizeof( Duple(3.0, -15.0) )

In [ ]:
# What happens here?
Duple(1.5, 3)

In [ ]:
struct Truple3{T,S}
    x::Tuple{T,S}
    z::S
end

We can also restrict the type parameter `T`:

In [ ]:
typeof("S") <: Number

In [ ]:
typeof(4) <: Number

In [ ]:
# T can be any subtype of Number, but nothing else
struct PlanarCoordinate{T<:Number}
    x::T
    y::T
end

In [ ]:
PlanarCoordinate("4th Ave", "14th St")

In [ ]:
PlanarCoordinate(2//3, 8//9)

Arrays are an exemple of mutable, parameterized types

### Why Use Types?

You can write all your code without thinking about types at all. If you
do this, however, you’ll be missing out on some of the biggest benefits
of using Julia.

If you understand types, you can:

-   Write faster code
-   Write expressive, clear, and well-structured programs (keep this in
    mind when we talk about functions)
-   Reason more clearly about how your code works

Even if you only use built-in functions and types, your code still takes
advantage of Julia’s type system. That’s why it’s important to
understand what types are and how to use them.

In [ ]:
# Example: writing type-stable functions
function sumofsins_unstable(n::Integer)  
    sum = 0:: Integer
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end  

function sumofsins_stable(n::Integer)  
    sum = 0.0 :: Float64
    for i in 1:n  
        sum += sin(3.4)  
    end  
    return sum 
end

# Compile and run
sumofsins_unstable(Int(1e5))
sumofsins_stable(Int(1e5))

In [ ]:
@time sumofsins_unstable(Int(1e5))

In [ ]:
@time sumofsins_stable(Int(1e5))

In `sumofsins_stable`, the compiler is guaranteed that `sum` is of type
`Float64` throughout; therefore, it saves time and memory. On the other
hand, in `sumofsins_unstable`, the compiler must check the type of `sum`
at each iteration of the loop. Let’s look at the LLVM [intermediate
representation](http://www.johnmyleswhite.com/notebook/2013/12/06/writing-type-stable-code-in-julia/).

### Multiple Dispatch

So far we have defined functions over argument lists of any type.
Methods allow us to define functions “piecewise”. For any set of input
arguments, we can define a **method**, a definition of one possible
behavior for a function.

In [ ]:
# Define one method of the function print_type
function print_type(x::Number)
    println("$x is a number")
end

In [ ]:
# Define another method
function print_type(x::String)
    println("$x is a string")
end

In [ ]:
# Define yet another method
function print_type(x::Number, y::Number)
    println("$x and $y are both numbers")
end

In [ ]:
# See all methods for a given function
methods(print_type)

Julia uses **multiple dispatch** to decide which **method** of a
**function** to execute when a function is applied. In particular, Julia
compares the types of *all* arguments to the signatures of the
function’s methods in order to choose the applicable one, not just the
first (hence “multiple”).

In [ ]:
print_type(5)

In [ ]:
print_type("foo")

In [ ]:
print_type([1, 2, 3])

#### Other types of functions

Julia supports a short function definition for one-liners

In [ ]:
f(x::Float64) = x^2.0
f(x::Int64) = x^3

As well as a special syntax for anonymous functions

In [ ]:
u->u^2

In [ ]:
map(u->u^2, [1,2,3,4])

### Keyword arguments and optional arguments

In [ ]:
f(a,b,c=true; algo="newton")

### Packing/unpacking

In [ ]:
t = (1,2,4)

In [ ]:
a,b,c = t

In [ ]:
[(1:10)...]

In [ ]:
cat([4,3], [0,1]; dims=1)

In [ ]:
l = [[4,3], [0,1], [0, 0], [1, 1]]
# how do I concatenate it ?

cat(l...; dims=1) ### see python's f(*s)

### Writing Julian Code

As we’ve seen, you can use Julia just like you use MATLAB and get faster
code. However, to write faster and *better* code, attempt to write in a
“Julian” manner:

-   Define composite types as logically needed
-   Write type-stable functions for best performance
-   Take advantage of multiple dispatch to write code that looks like
    math
-   Add methods to existing functions

### Just-in-Time Compilation

How is Julia so fast? Julia is just-in-time (JIT) compiled, which means
(according to [this StackExchange
answer](http://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do)):

> A JIT compiler runs after the program has started and compiles the
> code (usually bytecode or some kind of VM instructions) on the fly (or
> just-in-time, as it’s called) into a form that’s usually faster,
> typically the host CPU’s native instruction set. *A JIT has access to
> dynamic runtime information whereas a standard compiler doesn’t and
> can make better optimizations like inlining functions that are used
> frequently.*

> This is in contrast to a traditional compiler that compiles all the
> code to machine language before the program is first run.

In particular, Julia uses type information at runtime to optimize how
your code is compiled. This is why writing type-stable code makes such a
difference in speed!

## Additional Exercises

Taken from QuantEcon’s [Julia
Essentials](https://lectures.quantecon.org/jl/julia_essentials.html) and
[Vectors, Arrays, and
Matrices](https://lectures.quantecon.org/jl/julia_arrays.html) lectures.

1.  Consider the polynomial $$p(x) = \sum_{i=0}^n a_0 x^0$$ Using
    `enumerate`, write a function `p` such that `p(x, coeff)` computes
    the value of the polynomial with coefficients `coeff` evaluated at
    `x`.

2.  Write a function `solve_discrete_lyapunov` that solves the discrete
    Lyapunov equation $$S = ASA' + \Sigma \Sigma'$$ using the iterative
    procedure $$S_0 = \Sigma \Sigma'$$
    $$S_{t+1} = A S_t A' + \Sigma \Sigma'$$ taking in as arguments the
    $n \times n$ matrix $A$, the $n \times k$ matrix $\Sigma$, and a
    number of iterations.